In [1]:
import io
import requests
import PyPDF2

def split_pdf_pages(pdf_data):
    pages = []
    reader = PyPDF2.PdfFileReader(io.BytesIO(pdf_data))

    for page_number in range(reader.getNumPages()):
        if reader.getPage(page_number):
            pages.append(reader.getPage(page_number))

    return pages

In [7]:
urls = ["https://file.notion.so/f/s/aec659ea-4498-4366-913b-1623fe101bca/Visie_Duurzaam_Waalwijk_2030_-_3_december_2019.pdf?id=85c6f94c-daaa-45bc-815a-4e61136c26e8&table=block&spaceId=ca274338-dfb2-4f65-823d-2acde6195fea&expirationTimestamp=1684842163574&signature=fcoUtEmbprrJUu4fpzg4sV_DZiNaJbymfVwBOhovSwk&downloadName=Visie+Duurzaam+Waalwijk+2030+-+3+december+2019.pdf"]

In [8]:
import urllib.parse

def extract_title(url):
    parsed_url = urllib.parse.urlparse(url)
    query_params = urllib.parse.parse_qs(parsed_url.query)
    download_name = query_params.get("downloadName", [None])[0]
    return download_name

In [9]:
keywords = ["bouw", "groen", "groene"]

In [10]:
def count_keywords_in_string(string):
    keyword_count = 0
    for keyword in keywords:
        keyword_count += string.lower().count(keyword.lower())
    return keyword_count

In [11]:
for url in urls:
    response = requests.get(url)
    pdf_data = response.content
    pdf_pages = split_pdf_pages(pdf_data)
    for page_number, page in enumerate(pdf_pages):
        contents = page.extract_text()
        if contents =="":
            continue
        if not any(keyword in contents for keyword in keywords):
            continue
        if count_keywords_in_string(contents) <= 5:
            continue
        print("page number: ", page_number)
        print(page.extract_text())
        print("\n")

page number:  5
   
 
 
Visie Duurzaam Waalwijk 2030  6 Samenvatting  
De verandering van het klimaat en de gevolgen daarvan zijn urgente problemen en vragen om 
ambitieus en daadkrachtig ingrijpen.  
In de strategische Visie  ‘Samen werken aan de toekomst’ Waalwijk  2025 en het coalitieakkoord 
‘Samen Duurzaam Vooruit’ is het thema Duurzaamheid de “groene” draad. We willen 
klimaatneutraal zijn in 2043  en in 2030, dus al over tien jaar, halverwege op koers zijn . 
In deze notitie staat in hoofdlijnen de visie op Duurzaam Waalwijk 2030 uitgewerkt. De Visie  
Duurzaam Waalwijk kijkt tien jaar vooruit. De eerder beschreven doelen voor 2043 blijven 
onverminderd van kracht.  
De gemeente Waalwijk wil zich duurzaam ontwikkel en en overgaan naar een energieneutraal, CO 2-
arm en klimaatbestendig Waalwijk in 2043. Om dit te bereiken zijn er drie hoofdthema’s benoemd:  
• Energie   
De ambitie van Waalwijk is CO 2 arm te worden en de energie die we verbruiken duurzaam op te 
wekken. Concreet

In [14]:
import requests
from io import BytesIO
from PyPDF2 import PdfFileReader
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class PDFPageExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        extracted_pages = []
        for url in X:
            response = requests.get(url)
            pdf_data = response.content
            pdf_pages = self.split_pdf_pages(pdf_data)
            extracted_pages.extend(pdf_pages)
        return extracted_pages

    def split_pdf_pages(self, pdf_data):
        pdf = PdfFileReader(BytesIO(pdf_data))
        return [pdf.getPage(i) for i in range(pdf.getNumPages())]

class PageContentFilter(BaseEstimator, TransformerMixin):
    def __init__(self, keywords, max_keyword_count=5):
        self.keywords = keywords
        self.max_keyword_count = max_keyword_count

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        filtered_pages = []
        for page_number, page in enumerate(X):
            contents = page.extract_text()
            if contents == "":
                continue
            if not any(keyword in contents for keyword in self.keywords):
                continue
            if self.count_keywords_in_string(contents) <= self.max_keyword_count:
                continue
            filtered_pages.append((page_number, contents))
        return filtered_pages

    def count_keywords_in_string(self, text):
        count = 0
        for keyword in self.keywords:
            count += text.lower().count(keyword.lower())
        return count


# Create the pipeline
pipeline = Pipeline([
    ('pdf_extractor', PDFPageExtractor()),
    ('page_filter', PageContentFilter(keywords))

])

# Apply the pipeline to the URLs
filtered_pages = pipeline.transform(urls)

# Print the filtered pages
for page_number, contents in filtered_pages:
    print("Page number:", page_number)
    print(contents)
    print("\n")

Page number: 5
   
 
 
Visie Duurzaam Waalwijk 2030  6 Samenvatting  
De verandering van het klimaat en de gevolgen daarvan zijn urgente problemen en vragen om 
ambitieus en daadkrachtig ingrijpen.  
In de strategische Visie  ‘Samen werken aan de toekomst’ Waalwijk  2025 en het coalitieakkoord 
‘Samen Duurzaam Vooruit’ is het thema Duurzaamheid de “groene” draad. We willen 
klimaatneutraal zijn in 2043  en in 2030, dus al over tien jaar, halverwege op koers zijn . 
In deze notitie staat in hoofdlijnen de visie op Duurzaam Waalwijk 2030 uitgewerkt. De Visie  
Duurzaam Waalwijk kijkt tien jaar vooruit. De eerder beschreven doelen voor 2043 blijven 
onverminderd van kracht.  
De gemeente Waalwijk wil zich duurzaam ontwikkel en en overgaan naar een energieneutraal, CO 2-
arm en klimaatbestendig Waalwijk in 2043. Om dit te bereiken zijn er drie hoofdthema’s benoemd:  
• Energie   
De ambitie van Waalwijk is CO 2 arm te worden en de energie die we verbruiken duurzaam op te 
wekken. Concreet 